In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')

train.shape

In [ ]:
train.head()

In [ ]:
y_train = train['label'].values
X_train = train.drop(columns=['label']).values/255
X_test = test.values/255

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
fig, axes = plt.subplots(2,5, figsize=(12,5))
axes = axes.flatten() 
idx = np.random.randint(0,10,size=10) 
for i in range(10):
    axes[i].imshow(X_train[idx[i],:].reshape(28,28), cmap='gray')
    axes[i].axis('off') 
    axes[i].set_title(str(int(y_train[idx[i]])), color= 'black', fontsize=25) 
    plt.show()

In [ ]:
print(len(test))

In [ ]:
def relu(x):
    x[x<0]=0
    return x

In [ ]:
def h(X,W,b):
    a1 = X
    z1 = np.matmul(X, W[0]) + b[0]
    
    aa = relu(z1)
    zz = np.matmul(aa,W[1]) + b[1]
    
    a2 = relu(zz)
    z2 = np.matmul(a2, W[2])
    s = np.exp(z2)
    total = np.sum(s, axis=1).reshape(-1,1)
    sigma = s/total
    return sigma

In [ ]:
def softmax(X_in,weights):
    
    s = np.exp(np.matmul(X_in,weights))
    total = np.sum(s, axis=1).reshape(-1,1)
    return s / total

In [ ]:
def loss(y_pred,y_true):
    global K 
    K = 10
    N = len(y_true)
    y_true_one_hot_vec = (y_true[:,np.newaxis] == np.arange(K))
    loss_sample = (np.log(y_pred) * y_true_one_hot_vec).sum(axis=1)
    return -np.mean(loss_sample)

In [ ]:
def backprop(W,b,X,y,alpha=1e-4):
    K = 10
    N = X.shape[0]
    
    a1 = X
    z1 = np.matmul(X, W[0]) + b[0]
    
    aa = relu(z1)
    zz = np.matmul(aa, W[1]) + b[1]
    
    a2 = relu(zz)
    
    z2 = np.matmul(a2, W[2])
    s = np.exp(z2)
    total = np.sum(s, axis=1).reshape(-1,1)
    sigma = s/total
    
    y_one_hot_vec = (y[:,np.newaxis] == np.arange(K))
    delta2 = (sigma - y_one_hot_vec)
    grad_W1 = np.matmul(a2.T, delta2)
    
    delta_add = np.matmul(delta2, W[2].T)*(zz>0)
    grad_W_add = np.matmul(aa.T, delta_add)
    
    delta1 = np.matmul(delta_add,W[1].T)*(z1>0)
    grad_W0 = np.matmul(X.T, delta1)
    
    dW = [grad_W0/N + alpha*W[0], grad_W_add/N + alpha*W[1], grad_W1/N + alpha*W[2]]
    db = [np.mean(delta1, axis=0), np.mean(delta_add, axis=0)]
    return dW, db

In [ ]:
eta = 5e-1
alpha = 1e-6 # regularization
gamma = 0.99 # RMSprop
eps = 1e-3 # RMSprop
max_iter = 2000 # number of iterations of gradient descent
n_H = 256 # number of neurons in the hidden layer
n = X_train.shape[1] # number of pixels in an image
K = 10
s_h_y = 64

np.random.seed()
W = [1e-1*np.random.randn(n, n_H), 1e-1*np.random.randn(n_H, n_H), 1e-1*np.random.randn(n_H, K)]
b = [np.random.randn(n_H), np.random.randn(n_H)]
all_scores = []

In [ ]:
import time
err = 1e-5

In [ ]:
# def train_model(x_train,y_train):

#     score = 0
#     batch_size = 2000
#     num_batches = int((np.divide(x_train.shape[0], batch_size)))
#     print(num_batches)
#     X_batches = np.split(x_train,num_batches)
#     y_batches = np.split(y_train,num_batches)

#     time0 = time.time()
#     gW0 = gW1 = gb0 = gW2 = gb1 = 1

#     for i in range(2000):

#         for batch_index in range(0,num_batches):
#             dW, db = backprop(W,b,X_batches[batch_index],y_batches[batch_index],alpha)


#             gW0 = gamma*gW0 + (1-gamma)*np.sum(dW[0]**2)
#             etaW0 = eta/np.sqrt(gW0 + eps)
#             W[0] -= etaW0 * dW[0]

#             gW1 = gamma*gW1 + (1-gamma)*np.sum(dW[1]**2)
#             etaW1 = eta/np.sqrt(gW1 + eps)
#             W[1] -= etaW1 * dW[1]

#             gb0 = gamma*gb0 + (1-gamma)*np.sum(db[0]**2)
#             etab0 = eta/np.sqrt(gb0 + eps)
#             b[0] -= etab0 * db[0]


#             gW2 = gamma*gW2 + (1-gamma)*np.sum(dW[2]**2)
#             etaW2 = eta/np.sqrt(gW2 + eps)
#             W[2] -= etaW2 * dW[2]

#             gb1 = gamma*gb1 + (1-gamma)*np.sum(db[1]**2)
#             etab1 = eta/np.sqrt(gb1 + eps)
#             b[1] -= etab0 * db[1]
            
#             gW0 = gW1 = gb0 = gW2 = gb1 = 1


#         if i % 20 == 0:
            
#             y_pred = h(x_train,W,b)
#             new_score = np.mean(np.argmax(y_pred, axis=1)== y_train)
#             print("iter:score:loss ",i,round(new_score,5),round(loss(y_pred,y_train),5),end = "")
#             if new_score>0.999999:
#                 print()
#                 break
#             if new_score - score<=err:
#                 print("score reach max")
#                 break
#             score = new_score

#     y_pred_final = h(x_train,W,b)
#     print("Final training accuracy is {:.4%}".format(np.mean(np.argmax(y_pred_final, axis=1)== y_train)))


#     print("time: ", time.time()-time0,"s")
#     return W,b


# W = [1e-1*np.random.randn(n, n_H), 1e-1*np.random.randn(n_H, n_H), 1e-1*np.random.randn(n_H, K)]
# b = [np.random.randn(n_H), np.random.randn(n_H)]
# W,b = train_model(x_train,y_train)

In [ ]:
def update_W_b(dW,db):
    
    gW0 = gW1 = gb0 = gW2 = gb1 = 1
    
    gW0 = gamma*gW0 + (1-gamma)*np.sum(dW[0]**2)
    etaW0 = eta/np.sqrt(gW0 + eps)
    W[0] -= etaW0 * dW[0]

    gW1 = gamma*gW1 + (1-gamma)*np.sum(dW[1]**2)
    etaW1 = eta/np.sqrt(gW1 + eps)
    W[1] -= etaW1 * dW[1]

    gb0 = gamma*gb0 + (1-gamma)*np.sum(db[0]**2)
    etab0 = eta/np.sqrt(gb0 + eps)
    b[0] -= etab0 * db[0]


    gW2 = gamma*gW2 + (1-gamma)*np.sum(dW[2]**2)
    etaW2 = eta/np.sqrt(gW2 + eps)
    W[2] -= etaW2 * dW[2]

    gb1 = gamma*gb1 + (1-gamma)*np.sum(db[1]**2)
    etab1 = eta/np.sqrt(gb1 + eps)
    b[1] -= etab0 * db[1]

In [ ]:
%%time
def train_model(X_train,y_train,batch_size = 2000,iter_check = 50,alpha = 1e-4):
    score = 0
    num_batches = int(np.floor(np.divide(X_train.shape[0], batch_size)))
    X_batches = np.split(X_train[:(num_batches)*batch_size],num_batches)
    y_batches = np.split(y_train[:(num_batches)*batch_size],num_batches)

    time_last = time0 = time.time()
    
    for i in range(max_iter):
        for batch_index in range(0,num_batches):
            dW, db = backprop(W,b,X_batches[batch_index],y_batches[batch_index],alpha)
            update_W_b(dW,db)
    
        if time.time() - time_last > 5:
            time_last = time.time()
            print(i,end = " ")
            
        if i % iter_check == 0:
            # sanity check 1
            
            y_pred = h(X_train,W,b)
            new_score = np.mean(np.argmax(y_pred, axis=1)== y_train)
            
            if new_score - score<=err:
                print()
                print("score reach max",end = " ")
                print("Final iteration:",i,end = "")
                break
            score = new_score
            
    y_pred_final = h(X_train,W,b)
    print()
    print("Final cross-entropy loss is {:.8}".format(loss(y_pred_final,y_train)))
    print("Final training accuracy is {:.4%}".format(np.mean(np.argmax(y_pred_final, axis=1)== y_train)))
    print("time: ", time.time()-time0,"s")
    return W,b

W = [1e-1*np.random.randn(n, n_H), 1e-1*np.random.randn(n_H, s_h_y), 1e-1*np.random.randn(s_h_y, K)]
b = [np.random.randn(n_H), np.random.randn(s_h_y)]
W,b = train_model(X_train[:2000],y_train[:2000])

In [ ]:
%%time
# cross validation
k=6    # k fold
np.random.shuffle(np.array(train))

y_train = train['label'].values
X_train = train.drop(columns=['label']).values/255
fold_size = int(X_train.shape[0]/k)

scores = []
max_iter = 100
err = 1e-4
iter_check = 100

alpha_s = [x for x in range(2,8)]

for index in range(6):
    alpha = alpha_s[index]*1e-5
    s_h_y = 64
    batch_size = 2000
    W = [1e-1*np.random.randn(n, n_H), 1e-1*np.random.randn(n_H, s_h_y), 1e-1*np.random.randn(s_h_y, K)]
    b = [np.random.randn(n_H), np.random.randn(s_h_y)]
    X_train_temp = np.concatenate((X_train[0:index*fold_size],X_train[(index+1)*fold_size:]))
    y_train_temp = np.concatenate((y_train[0:index*fold_size],y_train[(index+1)*fold_size:]))
    X_test_temp = X_train[index*fold_size:(index+1)*fold_size]
    y_test_temp = y_train[index*fold_size:(index+1)*fold_size]
    W,b = train_model(X_train_temp, y_train_temp, batch_size = batch_size, iter_check = iter_check, alpha = alpha)
    score = np.mean(np.argmax(h(X_test_temp,W,b), axis=1)== y_test_temp)
    scores.append(score)
    print("CV for ",index," iteration: ",score)
    print()

print("cross validation score: ", end = "")
print(np.round(np.array(scores),4),np.argmax(scores))
scores.append(np.argmax(scores))
all_scores.append(scores)

In [ ]:
# predictions
y_pred_test = np.argmax(h(X_test,W,b), axis=1)

# Generating submission using pandas for grading
submission = pd.DataFrame({'ImageId': range(1,len(X_test)+1) ,'Label': y_pred_test })
submission.to_csv("submission.csv",index=False)

In [ ]:
from pandas import DataFrame

def create_csv(filename:str, data:DataFrame):
    directory = "Wrangled_Data/"
    path = os.path.join(directory, filename)
    
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    if os.path.exists(path):
        os.abcremove(path)
        
    data.to_csv(path, index=False)

In [ ]:
create_csv("train_new.csv", train)
create_csv("test_new.csv", test)